### Tryout model Pytorch

In [1]:
#!pip install pytorch-ignite
!pip install -e /workspace/VisionWorks

#!nvidia-smi               #shell ?
#!nvcc --version           #shell ?#
#!pip install tqdm
#!pip install sklearn
#!pip install wandb --upgrade

Obtaining file:///workspace/VisionWorks
  Attempting uninstall: visionworks
    Found existing installation: visionworks 0.1.0
    Can't uninstall 'visionworks'. No files were found to uninstall.
  Running setup.py develop for visionworks


In [2]:
import wandb
from ignite.contrib.handlers.wandb_logger import *
from ignite.contrib.engines.common import * # does contain the setup_wandb_logging
import os

import ignite
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix 
from ignite.handlers import ModelCheckpoint, EarlyStopping

from PIL import Image
from read_lapchol import LapcholDataset
from matplotlib.pyplot import imshow
#from transforms import ReadPILImage
import torch
from torch.optim import Adam
from torchvision.models import resnet18
from torchvision.models import resnet50
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import numpy as np

import random

import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import sklearn
from sklearn.model_selection import KFold

from visionworks.transformation.loading.image_loading import LoadPILImage
from visionworks.transformation.image.image_transforms import ResizePILImage, ResizeNumpyImage, CenterCropTorchvision, ResizeTorchvision 
from visionworks.transformation import TorchvisionTformWrapper

from torchvision import transforms
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor

from transforms import LabelToInt, LabelToTensor, DicttoImageandLabel 



In [3]:
wandb.login()
#!wandb login <13a1f666958082496c4b39a10519f6db055b76d3>
#!wandb login --relogin


wandb: Currently logged in as: jnjsurgery (use `wandb login --relogin` to force relogin)


True

In [4]:
wandb.init(project='project1', 
        entity='rubyegging',
    name="classify_gallbladder",
    config={  
        "learning_rate": 0.0001,
        "epochs": 5,
        "batch_size": 16,
        "classes": 2,
        "loss_function": "BCELoss",
        "optimizer": "Adam",
        "architecture": "Resnet50",
        "dataset": "Lapchol_dataset"},
    tags=["pytorch-ignite", "lapchol", "try-out"]
)

config = wandb.config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class_labels = [1,2]

# hyperparameters in config.  

# define transform
tform = Compose([LoadPILImage(fields=['frame_path']),ResizePILImage(fields=['frame_path'], width=256, height=256), 
                 CenterCropTorchvision(fields=['frame_path'], width=224, height=224),
                 TorchvisionTformWrapper(fields=['frame_path'], transform=ToTensor()), 
                 TorchvisionTformWrapper(fields=['frame_path'], transform=transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])),
                 LabelToInt(fields=['label'], labels=class_labels), LabelToTensor(fields=['label']) , 
                 DicttoImageandLabel(image=['frame_path'], label=['label']) ])
                 #])  #DicttoImageandLabel(image=['frame_path'], label=['label']

# Load lapchol dataset
print('The train set:')
lapchol_train = LapcholDataset(root_dir='/workspace/Lapchol_dataset', incl_videos=[88,89,92,94,95,97,98,102,103,105,106,107,109,110,111,113,115,118,119,120,125,127,139,144,150,152,156,160,172,180,189,205,214,215,220,224,226,227,228], sample_rate=1,transform=tform) #70% # 
print('Number of samples in train set:' + str(lapchol_train.__len__()))

print('The validation set:')
lapchol_val = LapcholDataset(root_dir='/workspace/Lapchol_dataset', incl_videos=[242,251,286,289,292,301,302,304,309,310,315,316,358,360,367,373,407,428,444], sample_rate=1, transform=tform) #20% )#
#lapchol_test = LapcholDataset(root_dir='D:/Ruby/Lapchol_dataset', incl_videos=[88,95], sample_rate=1) #10%
print('Number of samples in validation set:' + str(lapchol_val.__len__()))

# onderstaande nog even controleren @Fausto ? 
manualSeed = 1
def _init_fn():
    np.random.seed(manualSeed)
    
dataloader_train = DataLoader(lapchol_train, config.batch_size, shuffle=True, pin_memory=True, num_workers=0, worker_init_fn=_init_fn)
dataloader_val = DataLoader(lapchol_val, config.batch_size, shuffle=True, pin_memory=True, num_workers=0, worker_init_fn=_init_fn) # bij validation ook shuffle=True


The train set:
! Removed 0 samples with label excl
! Removed 803 samples with label 0
! Changed 4005 samples from 3 to 2
Number of samples in train set:9971
The validation set:
! Removed 0 samples with label excl
! Removed 147 samples with label 0
! Changed 465 samples from 3 to 2
Number of samples in validation set:2943


In [6]:
# Print the first sample, as an example (only possible without transform in Dataset)
#sample = lapchol_train.getdict()
#print(sample)
#print(sample[0].shape)     

#print(sample.__len__())

#print(sample)
#for i in range(0,sample.__len__()):
#    label = lapchol_train
    #print(label.__getitem__(i)[1])     




#for sample in samples:
#    label = samples[sample]['label']
#    print(label)

#pil_img = Image.open(sample['frame_path']).convert('RGB')
#imshow(np.asarray(pil_img))

In [7]:
loss = nn.L1Loss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

In [8]:
print(target)

tensor([[ 0.2460,  1.2294,  1.1120, -0.8817,  1.9755],
        [ 0.4889, -0.1875, -0.3292,  0.9594, -0.2271],
        [-0.0197,  0.9080, -0.9595,  0.2986, -1.5890]])


# Only possible without transforms in Dataset?
# how much frames for each label are there in lapchol_train?

total = 0
counter_dict = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
for i in range(lapchol_train.__len__()):
    sample = lapchol_train.__getitem__(i)
    label = sample.get('label')
    if label == 'excl':
        label = 4
    else:
        label = label
    counter_dict[int(label)] += 1
    total += 1

print(counter_dict)

for i in counter_dict:
    print(f"{i}: {counter_dict[i] / total * 100}")

total_nr_of_samples = lapchol_train.__len__()
print(f' Total number of samples in train set: {total_nr_of_samples}')

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = resnet50(pretrained=True)
num_ftrs = model.fc.in_features
#for binary classification:
model.fc = torch.nn.Sequential(torch.nn.Linear(num_ftrs, out_features=1,
                                               bias=True), nn.Sigmoid()) #output_features=2? num_ftrs is not the same as 512?
model.to(device)
#print(model)


#for multiple class classification
#model.fc = torch.nn.Sequential(torch.nn.Linear(num_ftrs, out_features=3,
#                                bias=True), torch.nn.Softmax(dim=1))

#for param in model.parameters():
#    param.requires_grad = False   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# beter goed gejat, dan slecht bedacht 
from ignite.metrics.metric import Metric
from ignite.exceptions import NotComputableError


class BinaryAccuracy(Metric):
    """
    Calculates the binary accuracy.

    - `update` must receive output of the form `(y_pred, y)`.
    - `y_pred` must be in the following shape (batch_size, ...) and it's elements must be between 0 and 1.
    - `y` must be in the following shape (batch_size, ...)
    """
    def reset(self):
        self._num_correct = 0
        self._num_examples = 0

    def update(self, output):
        y_pred, y = output
        correct = torch.eq(torch.round(y_pred).type(y.type()), y).view(-1)
        self._num_correct += torch.sum(correct).item()
        self._num_examples += correct.shape[0]

    def compute(self):
        if self._num_examples == 0:
            raise NotComputableError('BinaryAccuracy must have at least one example before it can be computed')
        return self._num_correct / self._num_examples

In [11]:

optimizer = Adam(model.parameters(), config.learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()
epochs = config.epochs

metrics = {'accuracy':BinaryAccuracy(), 'loss':Loss(criterion)} #'accuracy':Accuracy(), , 'cm':ConfusionMatrix(num_classes=len(class_labels))
   
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
train_evaluator = create_supervised_evaluator(model, metrics, device=device) 
val_evaluator = create_supervised_evaluator(model, metrics, device=device)    

log_interval =5


#from ignite.utils import setup_logger
#trainer.logger = setup_logger("Train Logger")
  
#training_history = {'accuracy':[],'loss':[]}
#validation_history = {'accuracy':[],'loss':[]}

#desc = "ITERATION - loss: {:.2f}"
#pbar = tqdm(initial=0, leave=False, total=len(loader), desc=desc.format(0))

@trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
def log_training_loss(engine):
    epoch = engine.state.epoch
    max_batches = engine.state.epoch_length
    batch_nr = engine.state.iteration
    batch_nr = batch_nr - (epoch - 1) * max_batches
    loss = engine.state.output
    wandb.log({"train loss": loss})
    #pbar.desc = desc.format(engine.state.output)
    #pbar.update(log_interval)
    print(f'Epoch[{epoch}] Batch[{batch_nr}/{max_batches}] Loss: {loss:.4f}')
    

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    #pbar.refresh()
    train_evaluator.run(dataloader_train)
    metrics = train_evaluator.state.metrics
    print(f"Training Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")
    wandb.log({"training loss": metrics['loss']})
    wandb.log({"training accuracy": metrics['accuracy']})


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    val_evaluator.run(dataloader_val)
    validation_acc = val_evaluator.state.metrics["accuracy"]
    #print("Epoch: {} Validation accuracy: {:.2f}".format(trainer.state.epoch, validation_acc))
    wandb.log({"validation loss": metrics['loss']})
    wandb.log({"validation accuracy": metrics['accuracy']})
    
    '''
    Using WandBLogger in ignite is a 2-step modular process: First, you need to create 
    a WandBLogger object. Then it can be attached to any trainer or evaluator to automatically log the metrics. 
    We'll do the following tasks sequentially: 1) Create a WandBLogger object and then
    1) Log training loss - attach to trainer
    2) Log validation loss - attach to evaluator
    3) Log optional Parameters
    4) Watch the model
    '''
#wandb_logger.attach_output_handler(
#trainer,
#event_name=Events.ITERATION_COMPLETED,
#tag="training",
#output_transform=lambda loss: {"loss": loss}
#)
 
#wandb_logger.attach_output_handler(
#val_evaluator,
#event_name=Events.EPOCH_COMPLETED,
#tag="validation",
#metric_names=["loss", "accuracy"],
#global_step_transform=lambda *_: trainer.state.iteration,
#)
 
#wandb_logger.attach_opt_params_handler(
#trainer,
#event_name=Events.ITERATION_STARTED,
#optimizer=optimizer,
#param_name='lr'  # optional
#)
 



In [12]:
wandb.watch(model,criterion,log=all,log_freq=10)

In [ ]:
trainer.run(dataloader_train, max_epochs=epochs)

Epoch[1] Batch[5/624] Loss: 0.5018
Epoch[1] Batch[10/624] Loss: 0.3810
Epoch[1] Batch[15/624] Loss: 0.2180
Epoch[1] Batch[20/624] Loss: 0.1451
Epoch[1] Batch[25/624] Loss: 0.0953
Epoch[1] Batch[30/624] Loss: 0.1743
Epoch[1] Batch[35/624] Loss: 0.1687
Epoch[1] Batch[40/624] Loss: 0.1045
Epoch[1] Batch[45/624] Loss: 0.1312
Epoch[1] Batch[50/624] Loss: 0.2402
Epoch[1] Batch[55/624] Loss: 0.1164
Epoch[1] Batch[60/624] Loss: 0.0458
Epoch[1] Batch[65/624] Loss: 0.1035
Epoch[1] Batch[70/624] Loss: 0.3127
Epoch[1] Batch[75/624] Loss: 0.2890
Epoch[1] Batch[80/624] Loss: 0.3840
Epoch[1] Batch[85/624] Loss: 0.0291
Epoch[1] Batch[90/624] Loss: 0.1459
Epoch[1] Batch[95/624] Loss: 0.2375
Epoch[1] Batch[100/624] Loss: 0.0170
Epoch[1] Batch[105/624] Loss: 0.0782
Epoch[1] Batch[110/624] Loss: 0.1284
Epoch[1] Batch[115/624] Loss: 0.0396


In [ ]:
torch.save(model(), 'workspace/Lapchol_dataset')